In [ ]:
!pip install pennylane pennylane-qiskit qiskit yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 15.

In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
data = pd.read_csv('nasdq.csv')

# Display the first few rows to ensure data is loaded correctly
print(data.head())

# Assuming the CSV has 'Open' and 'Close' columns (adjust column names if needed)
data['Change'] = data['Close'] - data['Open']
data['Target'] = data['Change'].apply(lambda x: 1 if x > 0 else 0)  # 1 if price went up, 0 if down

# Selecting the relevant columns and dropping any missing data
df = data[['Open', 'Close', 'Target']].dropna()

# Display the updated dataframe
print(df.head())


         Date      Open      High       Low     Close     Volume  \
0  2010-01-04  6.640000  6.810000  6.633333  6.746667  6514500.0   
1  2010-01-05  6.643333  6.773333  6.643333  6.766667  4445100.0   
2  2010-01-06  6.733333  6.786667  6.720000  6.763333  7340100.0   
3  2010-01-07  6.750000  6.766667  6.630000  6.673333  8498400.0   
4  2010-01-08  6.676667  6.766667  6.626667  6.743333  4347600.0   

   InterestRate  ExchangeRate    VIX  TEDSpread  EFFR         Gold        Oil  
0          0.11        1.4419  20.04       0.17  0.12  1117.699951  81.510002  
1          0.11        1.4402  19.35       0.18  0.12  1118.099976  81.769997  
2          0.11        1.4404  19.16       0.19  0.12  1135.900024  83.180000  
3          0.11        1.4314  19.06       0.20  0.10  1133.099976  82.660004  
4          0.11        1.4357  18.13       0.20  0.11  1138.199951  82.750000  
       Open     Close  Target
0  6.640000  6.746667       1
1  6.643333  6.766667       1
2  6.733333  6.763333

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df[['Open', 'Close']].values
y = df['Target'].values

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
import pennylane as qml
import numpy as np

# Set up quantum device
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Enhanced quantum circuit (variational layer)
def variational_circuit(params, x):
    # Use AngleEmbedding for data encoding
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))

    # Add multiple StronglyEntanglingLayers for better expressiveness
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))

    # Add an additional entangling layer (such as CNOT)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    # Additional rotation gates (you can experiment with different rotations)
    for i in range(n_qubits):
        qml.RY(x[0] * np.pi, wires=i)
        qml.RZ(x[1] * np.pi, wires=i)

    # Use a combination of different entanglement techniques
    qml.CNOT(wires=[0, 1])  # Entangling qubits 0 and 1

    # Optionally, use an additional feature map layer for re-uploading data
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))

# Initialize weights for the circuit
np.random.seed(42)
num_layers = 5
params = np.random.randn(num_layers, n_qubits, 3)

# QNode and cost function for binary classification
@qml.qnode(dev)
def quantum_neural_network(params, x):
    variational_circuit(params, x)  # Call the variational circuit
    return qml.expval(qml.PauliZ(0))  # This is the only measurement and return value

# Predictive function (unchanged)
def predict(params, X):
    predictions = [quantum_neural_network(params, x) for x in X]
    return np.sign(predictions)

# Cost function (unchanged)
def cost(params, X, y):
    predictions = [quantum_neural_network(params, x) for x in X]
    return np.mean((np.array(predictions) - y) ** 2)


In [ ]:
# from pennylane.optimize import AdamOptimizer

# # QNode and cost function for binary classification
# @qml.qnode(dev)
# def quantum_neural_network(params, x):
#     variational_circuit(params, x)
#     return qml.expval(qml.PauliZ(0))

# # Predictive function
# def predict(params, X):
#     predictions = [quantum_neural_network(params, x) for x in X]
#     return np.sign(predictions)

# # Cost function (Binary Cross-Entropy)
# def cost(params, X, y):
#     predictions = [quantum_neural_network(params, x) for x in X]
#     return np.mean((np.array(predictions) - y) ** 2)


In [ ]:
from sklearn.metrics import accuracy_score

# Predict on test data
y_pred = predict(params, X_test)
y_pred = [1 if p > 0 else 0 for p in y_pred]

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 48.79%


In [ ]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# import matplotlib.pyplot as plt

# # Predict on the test set
# y_pred = predict(params, X_test)
# y_pred = [1 if p > 0 else 0 for p in y_pred]  # Convert predictions to binary classification (0 or 1)

# # Compute the confusion matrix
# cm = confusion_matrix(y_test, y_pred)

# # Plot the confusion matrix
# plt.figure(figsize=(8, 6))
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Down', 'Up'])
# disp.plot(cmap=plt.cm.Blues, values_format='d')
# plt.title("Confusion Matrix for Stock Price Movement Prediction")
# plt.show()


In [ ]:
#from sklearn.metrics import classification_report

In [ ]:
# y_pred = predict(params, X_test)
# y_pred = [1 if p > 0 else 0 for p in y_pred]  # Convert predictions to 0 or 1

In [ ]:
# report = classification_report(y_test, y_pred, target_names=['Down', 'Up'])
# print(report)